In [ ]:
from XO import *
from MCTS_trainer import *

t = Trainer(Environment,100,100)
p, v = t.train(20,2)

In [ ]:
torch.save(policy_net.state_dict(), 'XO_policy_net.pth')
torch.save(value_net.state_dict(), 'XO_value_net.pth')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

plt.title("XO, Policy Network Losses")
plt.plot(np.arange(np.array(p).shape[0])+1,p)
plt.xlabel("Batches")
plt.ylabel("Loss")
plt.show()

plt.title("XO, Value Network Losses")
plt.plot(np.arange(np.array(v).shape[0])+1,v)
plt.xlabel("Batches")
plt.ylabel("Loss")
plt.show()

In [ ]:
game = Environment()
MC = MonteCarlo(board=game,num_simulations=1000)
game = Environment()
game.reset()

for i in range(9):
    print("---------- Player Move --------------")
    action = int(input())
    _,reward,done,_ = game.take_action(action)
    game.print()
    if done:
        break
        
    tree = MC.play_and_search(value_net,policy_net,game.game_state()[0],1,verbose=False)
    #action_probs = t.policy_net.forward(torch.tensor(game.game_state()[0]).type(torch.FloatTensor).reshape(1,-1)).reshape(-1).detach().cpu().numpy()
    
    print("---------- Agent Move --------------")
    action_probs = np.zeros(9)
    for action,child in tree.children.items():
        action_probs[action] += child.visit_count # Use Value as well or not?

    #action_probs = np.exp(action_probs/np.max(action_probs))
    action_probs *= game.get_valid_actions()
    action_probs /= np.sum(action_probs)
    best_action = np.argmax(action_probs)
    _,reward,done,_ = game.take_action(best_action)
    game.print()
    if done:
        break
    
    